# Visualization notebook

The goal of this notebook is to provide visualization of data using Kepler.gl


In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
import pandas as pd

In [ ]:
# --- Global variables

# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

In [ ]:
def to_gdf(path):
    """
    Parameters: 
        path: path of the file to read as a geodataframe
        
    return:
        a GeoDataFrame (with Geopandas) corresponding to the file path
    """
    gdf = gpd.GeoDataFrame.from_file(path)
    gdf = gdf.to_crs('epsg:4326')
    return gdf

## 1. Network data

## Aimsun map

In [ ]:
aimsun_path_regular = dbx + '/Private Structured data collection/Aimsun/Inputs/'
aimsun_path_complex = dbx + '/Private Structured data collection/Aimsun/Inputs_complex/'

aimsun_path = aimsun_path_complex

#detectors = to_gdf(aimsun_path +'detectors.shp')
# meterings = to_gdf(aimsun_path +'meterings.shp')
# centroids = to_gdf(aimsun_path +'centroids.shp')
# centroid_connections = to_gdf(aimsun_path +'centroid_connections.shp')
# nodes = to_gdf(aimsun_path +'nodes.shp')
# polygons = to_gdf(aimsun_path +'polygons.shp')
sections = to_gdf(aimsun_path +'sections.shp')
# sectionsGeo = to_gdf(aimsun_path +'sectionsGeo.shp')
# turnings = to_gdf(aimsun_path +'turnings.shp')

In [ ]:

fremont_map = KeplerGl(height=600)
# fremont_map.add_data(data = detectors, name="Detectors")
# fremont_map.add_data(data = meterings, name="meterings")
fremont_map.add_data(data = nodes, name="nodes")
# fremont_map.add_data(data = polygons, name="polygons")
fremont_map.add_data(data = sections, name="sections")
# fremont_map.add_data(data = sectionsGeo, name="sectionsGeo")
# fremont_map.add_data(data = turnings, name="turnings")
# fremont_map.add_data(data = centroids, name="centroids")
# fremont_map.add_data(data = centroid_connections, name="centroid_connections")
fremont_map

In [ ]:
print(sections['rd_type'].count())
print(sections['rd_type'].unique())
print("175: " + str(sections[sections['rd_type'] == 175.0]['id'].count()))
print("177: " + str(sections[sections['rd_type'] == 177.0]['id'].count()))
print("179: " + str(sections[sections['rd_type'] == 179.0]['id'].count()))
print("180: " + str(sections[sections['rd_type'] == 180.0]['id'].count()))
print("182: " + str(sections[sections['rd_type'] == 182.0]['id'].count()))
print("184: " + str(sections[sections['rd_type'] == 184.0]['id'].count()))
print("185: " + str(sections[sections['rd_type'] == 185.0]['id'].count()))

In [ ]:
print(nodes['nodetype'].unique())

print("0: " + str(nodes[nodes['nodetype'] == 0.0]['id'].count()))
print("1: " + str(nodes[nodes['nodetype'] == 1.0]['id'].count()))
print("2: " + str(nodes[nodes['nodetype'] == 2.0]['id'].count()))
print("3: " + str(nodes[nodes['nodetype'] == 3.0]['id'].count()))

## Traffic signals and stop signs

In [ ]:
def to_gdf_csv(path):
# https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html#from-wkt-format
    df = pd.read_csv(path)
    gdf = gpd.GeoDataFrame(
        df, crs='epsg:4326', geometry=gpd.points_from_xy(df.x, df.y))
    return gdf

In [ ]:
network_infra_path = data_path + "/Manual-made dataset (do not touch)/Network/Infrastructure/"

stop_signs = to_gdf_csv(network_infra_path + "Stop signs location/Stop_Signs.csv")
traffic_lights = to_gdf_csv(network_infra_path + "Traffic lights location/Traffic_Lights.csv")

In [ ]:
fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = stop_signs, name="Stop signs")
fremont_map.add_data(data = traffic_lights, name="Traffic lights")
fremont_map

In [ ]:
print(stop_signs.describe())

In [ ]:
from shapely.geometry import Point, LineString

path_taz = data_path + "/Data processing/Auxiliary files/Demand/OD demand/TAZ"
internal_taz = to_gdf(path_taz + "/Internal_TAZ.shp")
external_taz = to_gdf(path_taz + "/External_TAZ.shp")

# Get gravity centers for all TAZs (internal and external)
centroid_gravity = {}
for i in range(len(internal_taz['geometry'])):
    centroid_gravity[internal_taz['CentroidID'][i]] = internal_taz['geometry'][i].centroid
for i in range(len(external_taz['geometry'])):
    centroid_gravity[external_taz['CentroidID'][i]] = external_taz['geometry'][i].centroid

od_demand_path = data_path + "/Data processing/Temporary exports to be copied to processed data/Demand/OD demand/" 
internal_od = pd.read_csv(od_demand_path + "Internal OD grouped by timestamp.csv")
external_od = pd.read_csv(od_demand_path + "External OD grouped by timestamp.csv")

# Remove centroids with no demand
external_od = external_od[external_od["counts"] != 0]
internal_od = internal_od[internal_od["counts"] != 0]
internal_od = internal_od[internal_od["CentroidID_O"] != "int_16"]
internal_od = internal_od[internal_od["CentroidID_D"] != "int_16"]
internal_od = internal_od[internal_od["CentroidID_O"] != "int_62"]
internal_od = internal_od[internal_od["CentroidID_D"] != "int_62"]
# Fix analysis timestep at 6 PM
internal_od_6_pm = internal_od[internal_od["dt_15"]=="18:00"]
external_od_6_pm = external_od[external_od["dt_15"]=="18:0"]

internal_od_6_pm = internal_od_6_pm.reset_index()
external_od_6_pm = external_od_6_pm.reset_index()

external_demand = gpd.GeoDataFrame(columns=['CentroidID_O', 'CentroidID_D', "counts", 'geometry'])
for i in range(len(external_od_6_pm['CentroidID_O'])):
    origin_id = external_od_6_pm['CentroidID_O'][i]
    dest_id = external_od_6_pm['CentroidID_D'][i]
    demand = external_od_6_pm['counts'][i]
    external_demand.loc[i] = [origin_id, dest_id, demand, LineString([centroid_gravity[origin_id], centroid_gravity[dest_id]])]

internal_demand = gpd.GeoDataFrame(columns=['CentroidID_O', 'CentroidID_D', "counts", 'geometry'])
internal_external_demand = gpd.GeoDataFrame(columns=['CentroidID_O', 'CentroidID_D', "counts", 'geometry'])
for i in range(len(internal_od_6_pm['CentroidID_O'])):
    origin_id = internal_od_6_pm['CentroidID_O'][i]
    dest_id = internal_od_6_pm['CentroidID_D'][i]
    demand = internal_od_6_pm['counts'][i]
    if "ext" not in origin_id and "ext" not in dest_id:
        internal_demand.loc[i] = [origin_id, dest_id, demand, LineString([centroid_gravity[origin_id], centroid_gravity[dest_id]])]
    else:
        internal_external_demand.loc[i] = [origin_id, dest_id, demand, LineString([centroid_gravity[origin_id], centroid_gravity[dest_id]])]

print("Number of Vehicles Analyzed at 6 PM (Internal):", internal_od_6_pm.counts.sum())
print("Number of Vehicles Analyzed at 6 PM (External):", external_od_6_pm.counts.sum())
print("Number of Vehicles Analyzed at 6 PM (Total):", internal_od_6_pm.counts.sum() + external_od_6_pm.counts.sum())

fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = external_demand, name="External/External Demand")
fremont_map.add_data(data = internal_demand, name="Internal/Internal Demand")
fremont_map.add_data(data = internal_external_demand, name="Internal/External Demand")
fremont_map.add_data(data = internal_taz, name="Internal TAZs")
fremont_map.add_data(data = external_taz, name="External TAZs")
fremont_map

'''
fremont_map.save_to_html(file_name=data_path+"/Data processing/Kepler maps/Demand/6pm_demand_map.html")
frm_config = fremont_map.config
frm_map = KeplerGl(height=600, config=frm_config)
frm_map.add_data(data = external_demand, name="External/External Demand")
frm_map.add_data(data = internal_demand, name="Internal/Internal Demand")
frm_map.add_data(data = internal_external_demand, name="Internal/External Demand")
frm_map.add_data(data = internal_taz, name="Internal TAZs")
frm_map.add_data(data = external_taz, name="External TAZs")
frm_map
frm_map.save_to_html(file_name=data_path+"/Data processing/Kepler maps/Demand/6pm_demand_count_widths_map.html")
'''